In [1]:
!pip install --quiet arcgis

In [61]:
import os
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,Table;
import arcpy;
import urllib;
import shutil;
import tempfile;

In [62]:
ssurgo_url = r"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/DownloaderBasinsv2/FeatureServer/0/query";
ssurgo_lyr = FeatureLayer(ssurgo_url);

sdf = None;
for i in range (0,3000,1000):

    ssurgo_qry = ssurgo_lyr.query(
         where              = "1=1"
        ,out_fields         = 'huc_8,link'
        ,order_by_fields    = 'huc_8'
        ,return_geometry    = False
        ,return_all_records = False
        ,result_record_count = 1000
        ,result_offset = i
    );
    
    if sdf is None:
        sdf = ssurgo_qry.sdf;
    else:
        sdf = sdf.append(ssurgo_qry.sdf)
    
sdf.count()

HUC_8       2206
Link        2206
OBJECTID    2206
dtype: int64

In [63]:
src = r"F:\SitingStagingCache\EsriSSURGO\src";
if not os.path.exists(src):
    os.mkdir(src);
    
trg = r"F:\SitingStagingCache\EsriSSURGO\trg";
if not os.path.exists(trg):
    os.mkdir(trg);
    
tmp = tempfile.gettempdir();


In [64]:
for index, row in sdf.iterrows():
    fn = row.Link.split('/')[-1];
    
    if not os.path.exists(src + os.sep + fn):
        print("Downloading " + fn);
        urllib.request.urlretrieve(row.Link,src + os.sep + fn);
        

In [71]:
for index, row in sdf.iterrows():
    fn = row.Link.split('/')[-1];
    dr = os.path.splitext(fn)[0];
    
    if not os.path.exists(trg + os.sep + dr + '.gdb'):
        
        if os.path.exists(tmp + os.sep + dr):
            shutil.rmtree(tmp + os.sep + dr);
            
        os.mkdir(tmp + os.sep + dr);
        
        arcpy.ExtractPackage_management(
             in_package = src + os.sep + fn
            ,output_folder = tmp + os.sep + dr
            ,cache_package = "NO_CACHE"
        );
        
        zin = tmp + os.sep + dr + os.sep + 'v105' + os.sep + dr + '.gdb';
        
        if dr == 'lowerlittlearkansas,oklahoma_11140109':
            os.rename(
                 tmp + os.sep + dr + os.sep + 'v105' + os.sep + 'lowerlittlearkansas,oklahoma_11140109.gdb'
                ,tmp + os.sep + dr + os.sep + 'v105' + os.sep + 'LowerLittleArkansasOklahoma_11140109.gdb'
            );
            
        elif dr == 'Pacific':
            os.rename(
                 tmp + os.sep + dr + os.sep + 'v105' + os.sep + 'input_mapunits.gdb'
                ,tmp + os.sep + dr + os.sep + 'v105' + os.sep + 'Pacific.gdb'
            );

        shutil.move(
             zin
            ,trg
        );
        
        shutil.rmtree(tmp + os.sep + dr);
        